In [7]:
# Modifying the provided 'edited-model.py' to incorporate suggested improvements

# Modified script content with additional tuning, early stopping, and class distribution checks.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import joblib

In [8]:
# Load the data
df = pd.read_csv('C:/Users/honey/Documents/placment work/Electronical-AI-Device-Recogniser/khanya/data managment/datasets/acs-f2-dataset.csv')

In [9]:
# Data Preprocessing
df = df.drop('time', axis=1)

# Convert categorical labels to numeric
le = LabelEncoder()
df['equipment'] = le.fit_transform(df['equipment'])

# Split features and labels
X = df.drop('equipment', axis=1)
y = df['equipment']

# Analyze class distribution before SMOTE
print("Class distribution before SMOTE:")
print(y.value_counts())

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Handle class imbalance using SMOTE
sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X_scaled, y)

Class distribution before SMOTE:
equipment
3     9682
5     9388
7     9378
6     9348
2     9272
14    9155
13    9088
8     9015
11    9004
4     8849
12    8810
9     8784
10    8755
1     8688
0     8661
Name: count, dtype: int64


In [10]:
# Analyze class distribution after SMOTE
print("Class distribution after SMOTE:")
print(pd.Series(y_resampled).value_counts())

# One-hot encode the target labels
num_classes = len(np.unique(y))
y_resampled_onehot = to_categorical(y_resampled, num_classes=num_classes)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled_onehot, test_size=0.2, random_state=42)

Class distribution after SMOTE:
equipment
0     9682
1     9682
2     9682
3     9682
4     9682
5     9682
6     9682
7     9682
8     9682
9     9682
10    9682
11    9682
12    9682
13    9682
14    9682
Name: count, dtype: int64


In [11]:
# Define the model architecture with dropout
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),  # Increased dropout for stronger regularization
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),  # Added another dense layer with fewer neurons
    Dense(num_classes, activation='softmax')
])

# Compile the model with a lower learning rate for better generalization
model.compile(optimizer=Adam(learning_rate=0.0005), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test),
                    epochs=100, 
                    batch_size=32, 
                    callbacks=[early_stopping])







Epoch 1/100


c:\Users\honey\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3631/3631 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.3047 - loss: 2.0447 - val_accuracy: 0.4717 - val_loss: 1.5137
Epoch 2/100
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.4433 - loss: 1.5678 - val_accuracy: 0.5260 - val_loss: 1.3281
Epoch 3/100
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.4926 - loss: 1.4120 - val_accuracy: 0.5736 - val_loss: 1.2084
Epoch 4/100
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.5291 - loss: 1.3109 - val_accuracy: 0.5964 - val_loss: 1.1298
Epoch 5/100
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.5515 - loss: 1.2376 - val_accuracy: 0.6148 - val_loss: 1.0762
Epoch 6/100
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.5711 - loss: 1.1854 - val_accuracy: 0.6379 - val_loss: 1.0253
Epoch 7/100
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.5831 - loss: 1.1398 - val_accuracy: 0.6476 - val_loss: 0.9976
Epoch 8/100
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 16s 5ms/step - accuracy: 0.5955 - loss: 1.1161 - val

In [12]:

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

# Confusion matrix and classification report
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

print("Confusion Matrix:")
print(confusion_matrix(y_true_classes, y_pred_classes))

print("Classification Report:")
print(classification_report(y_true_classes, y_pred_classes))

908/908 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7678 - loss: 0.6541
Test Accuracy: 0.7666804194450378
908/908 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step
Confusion Matrix:
[[1387    7   21   17    2  227   51   22    4   31   20   10  112    3
    21]
 [   1 1856    1    2    0    0    0    0   12    0    0    0    0    0
     9]
 [   0    0  945   16    2  386  166  184   18    0    1    8  171    2
     0]
 [  10    0   25  983    0  554   62  239    2   59    2    0  120    2
     0]
 [   0    0    8    0 1876   15   12   16    0    0    0    0   14    5
     0]
 [   2    0   21   12    0 1700   37    7    0    2    0    0  125    0
     0]
 [   0    0   59    6    0  346 1288  114    5    1    0    0  110    3
     0]
 [   0    0   16   36    5  201   51 1618    0    0    0    0    5    1
     0]
 [  10    9   32    6    0    0   33    0 1769   13    0   47   21    0
    12]
 [  11    1   29   13    2  413   24    2    4 1321    0    8   91    0
    22]
 [  10    0    8    0    0  100 

In [13]:
# Save the trained model and the scaler for future use
model.save('new-model1.0.0.h5')
joblib.dump(scaler, 'new-scaler1.0.0.pkl')

['new-scaler1.0.0.pkl']

In [15]:
  # Return the new path so the user can download it